In [1]:
from allennlp.models.archival import load_archive
from chemdataextractor.data import find_data
import json
import pprint

c:\Users\jack_\miniconda3\envs\cde-dev-2\lib\site-packages\stanza\models\pos\trainer.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, 

In [5]:
overrides = {"model.text_field_embedder.token_embedders.bert.pretrained_model": find_data("models/scibert_cased_weights-1.0.tar.gz")}
cde_bert_archive = load_archive(find_data('models/bert_finetuned_crf_model-1.0a'), overrides=json.dumps(overrides))
cde_bertcrf_model = cde_bert_archive.model
cde_bertcrf_model_state_dict = cde_bertcrf_model.state_dict()

c:\Users\jack_\miniconda3\envs\cde-dev-2\lib\site-packages\pytorch_pretrained_bert\modeling.py:603: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(wei

In [13]:
cde_bertcrf_model.vocab._index_to_token

_IndexToTokenDefaultDict(None,
                         {'labels': {0: 'O', 1: 'I-CEM', 2: 'B-CEM'},
                          'tokens': {0: '@@PADDING@@', 1: '@@UNKNOWN@@'}})

In [31]:
import torch
import torch.nn as nn
from transformers import AutoModel, PreTrainedModel, AutoConfig, PretrainedConfig
from torch.utils.data import Dataset, DataLoader
from chemdataextractor.nlp.crf import ConditionalRandomField, allowed_transitions
from chemdataextractor.nlp.allennlp_modules import TimeDistributed
from chemdataextractor.errors import ConfigurationError
from typing import Dict, Optional, List, Tuple
from overrides import overrides

class BertCrfConfig(PretrainedConfig):
    model_type = 'bert'
    
    def __init__(
        self,
        num_tags: int = 3,
        dropout=0.1,
        label_namespace: str = "labels",
        label_encoding: Optional[str] = None,
        index_and_label: List[Tuple[int, str]] = None,
        constrain_crf_decoding: bool = None,
        include_start_end_transitions: bool = True,
        model_name_or_path: str = None,
        **kwargs
    ):
        self.num_tags = num_tags
        self.dropout = dropout
        self.label_namespace = label_namespace
        self.label_encoding = label_encoding
        self.index_and_label = index_and_label
        self.constrain_crf_decoding = constrain_crf_decoding,
        self.include_start_end_transitions = include_start_end_transitions,
        self.model_name_or_path = model_name_or_path
        super().__init__(**kwargs)


class BertCrfTagger(PreTrainedModel):
    config_class = BertCrfConfig  # Required for saving/loading
    
    def __init__(self, config):

        super().__init__(config)
        self.bert_model = AutoModel.from_config(AutoConfig.from_pretrained(config.model_name_or_path))
        self.num_tags = config.num_tags
        self.tag_projection_layer = TimeDistributed(
            nn.Linear(self.bert_model.config.hidden_size, self.num_tags)
        )
        
        self.label_encoding = config.label_encoding
        self.index_and_label = config.index_and_label
        self.index_to_label = self._index_to_label()
        self.label_to_index = self._label_to_index()
        
        if config.constrain_crf_decoding:
            if not config.label_encoding:
                raise ConfigurationError("constrain_crf_decoding is True, but "
                                         "no label_encoding was specified.")
            labels = self.index_to_label
            constraints = allowed_transitions(config.label_encoding, labels)
        else:
            constraints = None

        self.include_start_end_transitions = config.include_start_end_transitions
        self.crf = ConditionalRandomField(
                self.num_tags, constraints,
                include_start_end_transitions=config.include_start_end_transitions
        )

        
        # Dropout for regularization
        self.dropout = nn.Dropout(config.dropout)
    
    def _index_to_label(self):
        return {index: label for index, label in self.index_and_label}
    
    def _label_to_index(self):
        return {label: index for index, label in self.index_and_label}

    def forward(self, input_ids, attention_mask, labels=None):
        # BERT embeddings
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        
        # Project onto tag space
        logits = self.tag_projection_layer(sequence_output)
        best_paths = self.crf.viterbi_tags(logits, attention_mask)

        predicted_tags = [x for x, y in best_paths]

        output = {"logits": logits, "mask": attention_mask, "tags": predicted_tags}
        
        return output

    def decode(self, output_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        """
        Converts the tag ids to the actual tags.
        ``output_dict["tags"]`` is a list of lists of tag_ids,
        so we use an ugly nested list comprehension.
        """
        output_dict["tags"] = [
                [self.label_to_index[tag]
                 for tag in instance_tags]
                for instance_tags in output_dict["tags"]
        ]
        return output_dict


In [32]:
bertcrf_config = BertCrfConfig(
    num_tags=3,
    label_namespace="labels",
    label_encoding="BIO",
    index_and_label=[(0, "O"), (1, "B-CEM"), (2, "I-CEM")],
    constrain_crf_decoding=True,
    include_start_end_transitions=False,
    dropout=0.1,
    model_name_or_path="allenai/scibert_scivocab_cased"
)

In [33]:
bertcrf_config.save_pretrained(("C:/Users/jack_/Desktop/cde_models/hf_bert_crf_tagger"))

In [34]:
bertcrf_tagger = BertCrfTagger(bertcrf_config)

In [35]:
bertcrf_tagger

BertCrfTagger(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [36]:
state_dict = {}
for k, v in cde_bertcrf_model_state_dict.items():
    if k.startswith("text_field_embedder.token_embedder_bert"):
        state_dict[k[40:]] = v
    else:
        state_dict[k] = v
print(state_dict.keys())

dict_keys(['bert_model.embeddings.word_embeddings.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.self.query.weight', 'bert_model.encoder.layer.0.attention.self.query.bias', 'bert_model.encoder.layer.0.attention.self.key.weight', 'bert_model.encoder.layer.0.attention.self.key.bias', 'bert_model.encoder.layer.0.attention.self.value.weight', 'bert_model.encoder.layer.0.attention.self.value.bias', 'bert_model.encoder.layer.0.attention.output.dense.weight', 'bert_model.encoder.layer.0.attention.output.dense.bias', 'bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.intermediate.dense.weight', 'bert_model.encoder.layer.0.intermediate.dense.bias', 'bert_model.encoder.layer.0.output.dense.weight', 'bert_model

In [37]:
bertcrf_tagger.load_state_dict(state_dict=state_dict, strict=False)

_IncompatibleKeys(missing_keys=['crf.start_transitions', 'crf.end_transitions'], unexpected_keys=[])

In [38]:
bertcrf_tagger.save_pretrained("C:/Users/jack_/Desktop/cde_models/hf_bert_crf_tagger")

In [84]:
my_config = AutoConfig.from_pretrained("C:/Users/jack_/Desktop/cde_models/hf_bert_crf_tagger")
model = BertCrfTagger(config=my_config, vocab=cde_bertcrf_model.vocab, label_encoding="BIO", constrain_crf_decoding=True, include_start_end_transitions=False)

Some weights of the model checkpoint at C:/Users/jack_/Desktop/cde_models/hf_bert_crf_tagger were not used when initializing BertModel: ['crf._constraint_mask', 'crf.transitions', 'tag_projection_layer._module.bias', 'tag_projection_layer._module.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
